In [1]:
!pip install haversine


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import haversine as hv
import time

def haversine_dist(x, y):
    return hv.haversine(x, y, unit=hv.Unit.KILOMETERS)

def create_data_model(size):
    """Stores the data for the problem."""
    data = {}
    coord = pd.read_csv('tsp_input.csv')[:size]
    coord.set_index('Place_Name', inplace=True)
    data['distance_matrix'] = [[haversine_dist((coord.loc[i]['Latitude'],coord.loc[i]['Longitude']), (coord.loc[j]['Latitude'],coord.loc[j]['Longitude'])) for j in coord.index] for i in coord.index]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(result,size,sol_time, manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    result.loc[len(result['No. of cities'])] = [size ,route_distance,sol_time]



def main():
    result = pd.DataFrame(columns = ('No. of cities','Route Distance','Solution Time'))
    sizes = [10,20,30,40,50,70,100]
    for size in sizes:
        """Entry point of the program."""
        # Instantiate the data problem.
        data = create_data_model(size)

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)


        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        solver_start_time = time.time()

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        solver_end_time = time.time()

        sol_time = solver_end_time - solver_start_time

        # Print solution on console.
        if solution:
            print_solution(result,size,sol_time,manager, routing, solution)

    result.to_csv('ortools_TSP_results.csv')

if __name__ == '__main__':
    main()



Objective: 4396 miles
Route for vehicle 0:
 0 -> 6 -> 2 -> 9 -> 3 -> 4 -> 1 -> 8 -> 7 -> 5 -> 0

Objective: 5798 miles
Route for vehicle 0:
 0 -> 18 -> 16 -> 17 -> 5 -> 12 -> 11 -> 7 -> 8 -> 13 -> 1 -> 4 -> 14 -> 19 -> 3 -> 9 -> 2 -> 15 -> 6 -> 10 -> 0

Objective: 7409 miles
Route for vehicle 0:
 0 -> 18 -> 16 -> 17 -> 21 -> 10 -> 23 -> 6 -> 15 -> 2 -> 9 -> 3 -> 27 -> 19 -> 14 -> 4 -> 1 -> 13 -> 28 -> 8 -> 7 -> 29 -> 11 -> 12 -> 25 -> 26 -> 24 -> 5 -> 22 -> 20 -> 0

Objective: 8391 miles
Route for vehicle 0:
 0 -> 20 -> 32 -> 22 -> 35 -> 18 -> 16 -> 17 -> 30 -> 5 -> 24 -> 26 -> 33 -> 39 -> 25 -> 12 -> 31 -> 11 -> 29 -> 7 -> 8 -> 28 -> 13 -> 1 -> 4 -> 14 -> 34 -> 19 -> 27 -> 3 -> 9 -> 38 -> 2 -> 15 -> 6 -> 23 -> 10 -> 21 -> 37 -> 36 -> 0

Objective: 9607 miles
Route for vehicle 0:
 0 -> 10 -> 23 -> 6 -> 15 -> 2 -> 38 -> 9 -> 3 -> 44 -> 30 -> 27 -> 19 -> 34 -> 49 -> 14 -> 4 -> 1 -> 13 -> 45 -> 28 -> 8 -> 7 -> 42 -> 48 -> 5 -> 43 -> 29 -> 11 -> 31 -> 12 -> 25 -> 39 -> 33 -> 41 -> 46 -> 26